In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from keras import models
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam
from keras.utils import to_categorical

In [2]:
# Load data
path = 'C:/Users/akshg/Desktop/Emotion CNN/'
data = pd.read_csv('icml_face_data.csv')

In [ ]:
data.head()

In [ ]:
data['emotion'].unique()

In [ ]:
# Drop 'surprise' and 'disgust' labels
data = data[(data['emotion'] != 5) & (data['emotion'] != 1)]

In [ ]:
data['emotion'].unique()

In [ ]:
# Update emotions dictionary
emotions = {0: 'Angry', 1: 'Fear', 2: 'Happy', 3: 'Sad', 4: 'Neutral'}

In [ ]:
# Function to prepare data
def prepare_data(data):
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
    return image_array, image_label

In [ ]:
# Prepare data splits
train_image_array, train_image_label = prepare_data(data[data[' Usage']=='Training'])
val_image_array, val_image_label = prepare_data(data[data[' Usage']=='PrivateTest'])
test_image_array, test_image_label = prepare_data(data[data[' Usage']=='PublicTest'])

In [ ]:
# Preprocess data
train_images = train_image_array.reshape((train_image_array.shape[0], 48, 48, 1)).astype('float32') / 255
val_images = val_image_array.reshape((val_image_array.shape[0], 48, 48, 1)).astype('float32') / 255
test_images = test_image_array.reshape((test_image_array.shape[0], 48, 48, 1)).astype('float32') / 255

# Remove labels for 'surprise' and 'disgust'
train_labels = to_categorical(train_image_label, num_classes=7)[:, [0, 2, 3, 4, 6]]
val_labels = to_categorical(val_image_label, num_classes=7)[:, [0, 2, 3, 4, 6]]
test_labels = to_categorical(test_image_label, num_classes=7)[:, [0, 2, 3, 4, 6]]

In [ ]:
# Define CNN model
model = models.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPool2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPool2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')  # Updated to 5 classes
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

In [ ]:
# Train the model
history = model.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    epochs=15,
                    batch_size=32)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

In [ ]:
# Plot training/validation loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot training/validation accuracy
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
pred_test_labels = model.predict(test_images)

In [ ]:
# Plot comparison of distributions
def plot_distribution_comparison(true_labels, pred_labels, title1='', title2=''):
    fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharey=False)
    x = emotions.values()
    
    true_counts = pd.Series(true_labels.argmax(axis=1)).value_counts().sort_index()
    pred_counts = pd.Series(pred_labels.argmax(axis=1)).value_counts().sort_index()
    
    axs[0].bar(x, true_counts, color='orange', label='True Distribution')
    axs[0].set_title(title1)
    axs[0].legend()
    
    axs[1].bar(x, pred_counts, label='Predicted Distribution')
    axs[1].set_title(title2)
    axs[1].legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot comparison of true and predicted distributions for train and validation sets
plot_distribution_comparison(train_labels, val_labels, title1='True Train Distribution', title2='Predicted Train Distribution')
plot_distribution_comparison(test_labels, pred_test_labels, title1='True Test Distribution', title2='Predicted Test Distribution')

In [ ]:
# from sklearn.model_selection import GridSearchCV

# # Define hyperparameters grid
# param_grid = {
#     'learning_rate': [0.001, 0.01, 0.1],
#     'batch_size': [32, 64, 128],
#     'epochs': [10, 15, 20]
# }

# # Create KerasClassifier wrapper for use with GridSearchCV
# model = KerasClassifier(build_fn=create_model)

# # Perform grid search
# grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1)
# grid_result = grid.fit(train_images, train_labels)

# # Summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

# Load pre-trained VGG16 model without top layers (include_top=False)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of VGG16
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)  # Output layer with 5 classes

# Combine base model with custom layers
model_vgg16 = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model_vgg16.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_vgg16 = model_vgg16.fit(train_images, train_labels,
                                validation_data=(val_images, val_labels),
                                epochs=15,
                                batch_size=32)

# Evaluate the model
test_loss_vgg16, test_acc_vgg16 = model_vgg16.evaluate(test_images, test_labels)
print('VGG16 Test accuracy:', test_acc_vgg16)


In [ ]:
from keras.applications import ResNet50

# Load pre-trained ResNet50 model without top layers (include_top=False)
base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Freeze the base model layers
for layer in base_model_resnet.layers:
    layer.trainable = False

# Add custom classification layers on top of ResNet50
x_resnet = base_model_resnet.output
x_resnet = Flatten()(x_resnet)
x_resnet = Dense(256, activation='relu')(x_resnet)
predictions_resnet = Dense(5, activation='softmax')(x_resnet)  # Output layer with 5 classes

# Combine base model with custom layers
model_resnet = Model(inputs=base_model_resnet.input, outputs=predictions_resnet)

# Compile the model
model_resnet.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_resnet = model_resnet.fit(train_images, train_labels,
                                  validation_data=(val_images, val_labels),
                                  epochs=15,
                                  batch_size=32)

# Evaluate the model
test_loss_resnet, test_acc_resnet = model_resnet.evaluate(test_images, test_labels)
print('ResNet Test accuracy:', test_acc_resnet)
